In this notebook we will:
- select a jet in the Solar Jet Hunter catalogue
- plot a frame of the jet and plot the average box around the jet
- plot all the boxes found from different subjects in Zooniverse for this jet
- plot one of the subject box on the jet

You will need to have the data already downloaded for this jet, see the notebook 'SolarJetHunter_download_solar_data'

# Open catalogue and select a jet

### Open catalogue
See tutorial for more details

In [3]:
import numpy as np
from utils.Jet_class_light import json_import_list

In [13]:
json_file = 'exports/Jet_clusters_3.0_2.0.paperID.json'
Jet_catalogue=json_import_list(json_file)

The 883 JetCluster objects are imported from exports/Jet_clusters_3.0_2.0.paperID.json.


### Select a jet

In [14]:
jetid = 508
selected_jet = Jet_catalogue[jetid]

Print some jet properties as a check

In [15]:
jet_start_time = selected_jet.obs_time
jet_end_time = selected_jet.obs_time + np.timedelta64(int(selected_jet.Duration*60), 's')
jet_sjh_identifier = selected_jet.ID
print(jet_start_time)
print(jet_end_time)
print(jet_sjh_identifier)

2015-09-21T02:00:30.000000
2015-09-21T02:45:06.000000
sjh_2015-09-21T02_1


# Plot one frame of the jet and overlay average box

# Plot all subject boxes and average box

# Plot one subject box over the jet